[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/nepslor/B5203E-TSAF/blob/main/W5/exponential_smoothing_implementation_solutions.ipynb)

# Exponential smoothing implementation
In this exercise we will try to craft different flavours of exponential smoothing algorithms from scratch, and compare their performance.



In [ ]:
%%capture
!pip install wget
import pandas as pd
import wget
import numpy as np
data = pd.read_pickle(wget.download("https://zenodo.org/record/4549296/files/reduced_dataset.pk?download=1"));

In [ ]:
samples_per_day = 24
data = pd.concat([data['all'], pd.Series(np.vstack(data['ghi_backwards'])[:, 0], name='ghi', index=data.index), pd.Series(np.vstack(data['temperature'])[:, 0], name='T', index=data.index)], axis=1)
data = data.resample('1h', origin='start').mean()
data /= data.std()
data = data.iloc[48:]
data.head()



In [ ]:
import matplotlib.pyplot as plt
import statsmodels.api as sm
fig, ax = plt.subplots(2, 1, figsize=(10, 4))
_ = sm.graphics.tsa.plot_acf(data['all'], lags=24*7, ax=ax[0])

y = data['all'].loc[data.index<'2019-01-13']
y = pd.concat([y, pd.Series(y.index.dayofyear, name='day', index=y.index),
               pd.Series(y.index.hour, name='hour', index=y.index)], axis=1)
y_mat = y.pivot(index='day', columns='hour', values='all').T
ax[1].matshow(y_mat, aspect='auto')



From the ACF plot it is clear that the signal is strongly seasonal, with a period of 24 hours. The heatmap plot also shows that the signal has a daily seasonality, with higher values during the day and lower values during the night. Let's keep a seasonality = 24 for our models.

In the following we just define an auxiliary function for showing some animations. It accepts a pd.DataFrame of ground truth values and a numpy matrix of predictions


In [ ]:
#@title Animation
from matplotlib import animation
from IPython.display import HTML

def ts_animation(y_te, y_hat, n_rows=50, labels=None):
  "plot the first n_rows of the two y_te and y_hat matrices"
  if labels is None:
    labels = ['1', '2']
  fig, ax = plt.subplots(1);
  y_min = np.minimum(np.min(y_hat), np.min(y_te))
  y_max = np.maximum(np.max(y_hat), np.max(y_te))
  line1, = ax.plot(y_hat[0], lw=2, label=labels[0]);
  line2, = ax.plot(y_hat[0], lw=2, label=labels[1]);
  plt.legend()
  ax.set_ylim(y_min, y_max)
  n_sa = y_hat.shape[1]
  def animate(i):
    line1.set_data(np.arange(n_sa),y_te[i:i+n_sa]);
    line2.set_data(np.arange(n_sa),y_hat[i,:]);
    return (line1,line2)

  def init():
    line1.set_data([], []);
    return (line1,)

  ani = animation.FuncAnimation(fig, animate, init_func=init, frames=n_rows, interval=100,
                                blit=True)
  plt.close('all')
  #rc('animation', html='jshtml')
  return HTML(ani.to_jshtml())


 ## Simple exponential smoothing
The following code implements a simple exponential smoothing, with no trend nor seasonality:

$$\begin{aligned}
& \hat{y}_{t+h \mid t}=\ell_t \\
& \ell_t=\alpha y_t+(1-\alpha) \ell_{t-1}
\end{aligned}$$

❓ Try to see the effect of the parameter alpha on the model's forecast. What do you observe?


In [ ]:
def simple_smoothing(y, h=1, alpha=0.8):
  y_hat = y.iloc[0]
  for y_i in y.values:
    y_hat = alpha*y_i + (1-alpha)*y_hat
  return np.tile(y_hat, h)


y_hat = []
for i in range(100):
  y_hat.append(simple_smoothing(data['all'].iloc[:np.maximum(1, i)], 24))
y_hat = np.vstack(y_hat)

ts_animation(data['all'].iloc[:100], y_hat, labels=['ground truth', 'predictions'])


In [ ]:
y_hat = []
for i in range(100):
  y_hat.append(simple_smoothing(data['all'].iloc[:np.maximum(1, i)], 24, alpha=0.2))
y_hat = np.vstack(y_hat)

ts_animation(data['all'].iloc[:100], y_hat, labels=['ground truth', 'predictions'])

## Holt's linear trend
We can additionally model the trend to have a linear expression for the forecasts, as a function of the step ahead. This makes the model more expressive but it can also lead to over or undershoot the prediction for high step ahead. Try to explore the combinations of the $\alpha$ and $\beta$ parameters

\begin{aligned}
& \hat{y}_{t+h \mid t}=\ell_t+h b_t \\
& \ell_t=\alpha y_t+(1-\alpha)\left(\ell_{t-1}+b_{t-1}\right) \\
& b_t=\beta^*\left(\ell_t-\ell_{t-1}\right)+\left(1-\beta^*\right) b_{t-1}
\end{aligned}

In [ ]:
def holt_smoothing(y, h=1, alpha=0.8, beta=0.1):
  l, l_past = y.iloc[0], y.iloc[0]
  b = 0
  for y_t in y.values:
    l = alpha*y_t + (1-alpha)*(l_past+b)
    b = beta*(l-l_past) + (1-beta)*b
    l_past = l

  return l + b*np.arange(h)

In [ ]:
y_hat = []
for i in range(100):
  y_hat.append(holt_smoothing(data['all'].iloc[:np.maximum(1, i)], 24, alpha=0.9, beta=0.05))
y_hat = np.vstack(y_hat)

ts_animation(data['all'].iloc[:100], y_hat, labels=['ground truth', 'predictions'])

## Holt Winter
Since we have seen that the signal we are trying to forecast is strongly seasonal, we can try to model it using the Holt-Winter model, which also estimate a seasonal component.


\begin{aligned}
\hat{y}_{t+h \mid t} & =\ell_t+h b_t+s_{index + h} \\
\ell_t & =\alpha\left(y_t-s_{t-m}\right)+(1-\alpha)\left(\ell_{t-1}+b_{t-1}\right) \\
b_t & =\beta^*\left(\ell_t-\ell_{t-1}\right)+\left(1-\beta^*\right) b_{t-1} \\
s_{index} & =\gamma\left(y_t-\ell_{t-1}-b_{t-1}\right)+(1-\gamma) s_{index}
\end{aligned}
where $index = t \mod m$ and $m$ is the period of the seasonality. The seasonal component is stored in a vector $s$ of length $m$. The parameters $\alpha$, $\beta$ and $\gamma$ are the smoothing parameters for the level, trend and seasonality respectively.
### ❓ HW model
Try to complete the estimation for the seasonal components used by the HW method. As you can see from the code, $s$ is a vector containing the estimated values for the seasonal profile.

In [ ]:

def holt_winters(y, s_init=None, h=1, alpha=0.8, beta=0.1, gamma=0.1, m=24, return_s=False):
  """
  h: steps ahead to be predicted
  m: period of the seasonality
  """
  l, l_past = 0, 0
  s = s_init.copy() if s_init is not None else np.zeros(m)
  b = 0
  for t, y_t in enumerate(y):
    index = t%m
    s[index] = gamma*(y_t-l_past-b) + (1-gamma)*s[index-m]
    l = alpha*(y_t-s[index]) + (1-alpha)*(l_past+b)
    b = beta*(l-l_past) + (1-beta)*b
    l_past = l

  # roll the seasonal component and take just the relevant part
  seasonal = np.roll(s, -index)[:h]

  # prediction equation
  preds = l + b*np.arange(h) + seasonal

  if return_s:
    return preds, s
  else:
    return preds



Let's see the model's prediction when all states are initialized with zero values:

In [ ]:
m = 24

y_hat = []
seasonal_state = []
for i in range(600):
  preds, s = holt_winters(data['all'].iloc[:np.maximum(1, i)].copy().values, h=24, alpha=0.1, beta=0.01, gamma=0.2, m=24, return_s=True)
  y_hat.append(preds)
  seasonal_state.append(s)

y_hat = np.vstack(y_hat)
seasonal_state = np.vstack(seasonal_state)


ts_animation(data['all'].values, y_hat, 100, labels=['ground truth', 'predictions'])


The next cell shows the HW's "learning" of the seasonality state vector. It seems that initializing this state could be helpful.

In [ ]:
ts_animation(data['all'].values, seasonal_state, 200, labels=['ground truth', 'seasonal component'])

In the next cell we use the s_init argument to pass the first day of observations to the model, to initialize the values in s

In [ ]:
m = 24

y_hat = []
s_init = data['all'].iloc[:m].copy().values

for i in range(600):
  y_hat.append(holt_winters(data['all'].iloc[:np.maximum(1, i)].values, s_init=s_init,  h=24, alpha=0.1, beta=0.01, gamma=0.2, m=24))
y_hat = np.vstack(y_hat)

ts_animation(data['all'].values, y_hat, 150)

# ❓ Optimal parameters
Let's try to find the optimal parameters for the Holt winter model. You can use any optimization technique to find the optimal values.
So far we didn't *fit* the model's parameters, we just set them manually. Let's try to find the optimal values for the parameters $\alpha$, $\beta$ and $\gamma$ by minimizing the RMSE over a training set. We will then test the model on a test set.
### Scoring strategy
We could just compute the RMSE on one forecasting window (24 hours) and tune the parameters to minimize this error. However, *this will lead to overfitting the parameters to a specific window*. In this course is severely forbidden 😉!

To have a more robust estimate of the parameters, we will use a rolling forecasting origin strategy with a *fit once* procedure:
1) First, we fit the model once on a training set
2) We will compute the RMSE over multiple forecasting windows on a test set, and then average the RMSEs to get a more robust estimate of the model's performance.

<img src="https://raw.githubusercontent.com/nepslor/B5203E-TSAF/main/pics/time-series-backtesting-forecasting-no-refit.gif" width="600">

In [ ]:
tr_ratio = 0.2
n_tr = int(len(data)*tr_ratio)
df_tr = data['all'].iloc[:n_tr]
df_te = data['all'].iloc[n_tr:]

In [ ]:

def get_scores(y, n_steps, n_sa, alpha, beta, gamma, m=24):
  y_hat = []
  y_np = y.values.ravel()
  scores = np.zeros(n_steps)
  s_init = data['all'].iloc[:m].values
  # What does this code do?
  for i in range(n_steps):
    y_hat = holt_winters(np.copy(y_np[:np.maximum(1, i)]), s_init,  n_sa, alpha=alpha, beta=beta, gamma=gamma, m=m)
    errs = y_hat-y_np[np.maximum(1, i):np.maximum(1, i)+n_sa]
    scores[i] = np.mean(errs**2)**0.5
  return np.mean(scores)


from tqdm import tqdm
n_trials = 500
pars = np.random.rand(n_trials, 3)
scores = np.zeros(n_trials)
print('start fitiing')
for n in tqdm(range(n_trials)):
    alpha, beta, gamma = pars[n, :]
    scores[n] = get_scores(df_tr, 500, 24, alpha, beta, gamma)

We can now have a look at the predictions of the best model

In [ ]:
best_pars = pars[np.argmin(scores), :]
print('best pars: {}, {} ,{}'.format(*best_pars))

y_hat = []
y_np = df_te.values.ravel()
s_init = df_te.iloc[:m].copy().values
for i in range(600):
  y_hat.append(holt_winters(y_np[:np.maximum(1, i)], s_init, 24, alpha=best_pars[0], beta=best_pars[1], gamma=best_pars[2]))
y_hat = np.vstack(y_hat)
ts_animation(df_te.iloc[200:], y_hat[200:, :], 200)

In [ ]:
plt.plot(np.sort(scores))
plt.xlabel('parameter tested sorted by score')
plt.ylabel('RMSE')

## Modified Holt Winters
In the following cell we introduce an additional state, the last step error made by the HW forecaster. This error is subtracted for the estimation of the level and seasonality, and added to the prediction with an exponential decreasing influence over the prediction horizon

In [ ]:
def modified_holt_winters(y, s_init=None, h=1, alpha=0.8, beta=0.1, gamma=0.1, omega=0.9, m=24, return_s=False):
  """
  h: steps ahead to be predicted
  m: period of the seasonality
  """
  l, l_past = 0, 0
  s = s_init.copy() if s_init is not None else np.zeros(m)
  b = 0
  eps = 0
  for t, y_t in enumerate(y):
    index = t%m
    s[index] = gamma*(y_t-l_past-b-eps) + (1-gamma)*s[index]
    l = alpha*(y_t-s[index]-eps) + (1-alpha)*(l_past+b)
    b = beta*(l-l_past) + (1-beta)*b
    eps = omega * (y_t-l_past-b-s[index]) + (1-omega) * eps
    l_past = l

  preds = l + b*np.arange(h) + np.hstack([s[index:], s[:index]])[:h] + eps*(h-np.arange(h))**2/h**2
  if return_s:
    return preds, s
  else:
    return preds


m = 24

y_hat = []
seasonal_state = []
s_init = data['all'].iloc[:m].copy().values
for i in range(600):
  preds, s = modified_holt_winters(data['all'].iloc[:np.maximum(1, i)].copy().values, h=24, alpha=best_pars[0], beta=best_pars[1], gamma=best_pars[2], m=24, return_s=True, s_init=s_init)
  y_hat.append(preds)
  seasonal_state.append(s)

y_hat = np.vstack(y_hat)
seasonal_state = np.vstack(seasonal_state)


ts_animation(data['all'].values[200:], y_hat[200:, :], 200, labels=['ground truth', 'predictions'])

# Extra: sensitivity analysis of the parameter importance

In [ ]:
df_scores = pd.DataFrame(np.hstack([pars,scores.reshape(-1, 1)]), columns=['alpha', 'beta', 'gamma', 'score']).sort_values('score')

fig, ax = plt.subplots(1, 3, figsize=(15, 4))
v_min = df_scores['score'].min()
v_max = df_scores['score'].max()
from matplotlib.colors import PowerNorm
norm = PowerNorm(gamma=0.1, vmin=v_min, vmax=v_max)
ax[0].scatter(df_scores['alpha'], df_scores['beta'], c=df_scores['score'], cmap='viridis', norm=norm, s=2)
ax[0].set_xlabel('alpha')
ax[0].set_ylabel('beta')
ax[1].scatter(df_scores['beta'], df_scores['gamma'], c=df_scores['score'], cmap='viridis', norm=norm, s=2)
ax[1].set_xlabel('beta')
ax[1].set_ylabel('gamma')
ax[2].scatter(df_scores['alpha'], df_scores['gamma'], c=df_scores['score'], cmap='viridis', norm=norm, s=2)
ax[2].set_xlabel('alpha')
ax[2].set_ylabel('gamma')


# Nixtla statsforecast implementation
Nixtla provides an efficient implementation of many forecasting models, including the AutoETS class: https://nixtlaverse.nixtla.io/statsforecast/src/core/models.html#autoets. Let's see how it performs on our dataset.
The AutoETS class automatically selects the best exponential smoothing model for the data, based on the AICc criterion. The class search among the exponential smoothing models based on three components: error, trend and seasonality. Each component can be additive (A), multiplicative (M) or none (N). The model is specified by a three letter string, where the first letter indicates the error type, the second letter the trend type and the third letter the seasonality type. For example, the model 'AAA' indicates an additive error, additive trend and additive seasonality. The model 'MNN' indicates a multiplicative error, no trend and no seasonality.
If the components are set to 'Z', the class will automatically select the best component type based on the AICc criterion. For example, the model 'ZZZ' indicates that the class will automatically select the best error, trend and seasonality types.


In [ ]:
from statsforecast import StatsForecast
from statsforecast.models import AutoETS
model = AutoETS(season_length=24, model='ZZZ')
fitted_model = model.fit(df_tr.values)


We can implement our *fit once* and rolling forecasting origin strategy by using the `.forward` method of the StatsForecast class. This method accepts as input the time series up to the last timestep, and the number of steps ahead to be predicted.

In [ ]:
y_hat = []
for i in range(200):
    y_past = np.hstack([df_tr.values, df_te.iloc[:i].values])
    preds = fitted_model.forward(y_past, h=24)
    y_hat.append(preds['mean'])
y_hat = np.vstack(y_hat)
ts_animation(df_te, y_hat, 200)